# Final TP

- XGBoost and AdaBoost

In [1]:
# % pip install xgboost

In [2]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Feature extraction

**Time-Based Features:**
- Duration of sleep cycles.
- Time spent in different sleep phases (light sleep, deep sleep, REM).
- Sleep onset latency (time taken to fall asleep).

**Frequency-Based Features:**
- Heart rate variability during different sleep phases.
- Respiratory rate.

**Statistical Features:**
- Mean, median, and standard deviation of various physiological signals during sleep.
- Signal entropy to capture complexity.

**Temporal Patterns:**
- Trends and patterns in sleep cycles over multiple nights.

**Previous Sleep History:**
- Consider features related to the sleep pattern from the previous night.

**Biometric Data:**
- If available, include biometric data like age, gender, and BMI.